<a href="https://colab.research.google.com/github/syedbabart/convLSTM-covid19/blob/master/newcases_covid19pak_convLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd

In [ ]:
df = pd.read_excel(r'/new_cases_Pakistan.xlsx')
df_deaths = pd.read_excel(r'/new_deaths_Pakistan.xlsx')

In [ ]:
df_deaths.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244
date,2019-12-31 00:00:00,2020-01-01 00:00:00,2020-01-02 00:00:00,2020-01-03 00:00:00,2020-01-04 00:00:00,2020-01-05 00:00:00,2020-01-06 00:00:00,2020-01-07 00:00:00,2020-01-08 00:00:00,2020-01-09 00:00:00,2020-01-10 00:00:00,2020-01-11 00:00:00,2020-01-12 00:00:00,2020-01-13 00:00:00,2020-01-14 00:00:00,2020-01-15 00:00:00,2020-01-16 00:00:00,2020-01-17 00:00:00,2020-01-18 00:00:00,2020-01-19 00:00:00,2020-01-20 00:00:00,2020-01-21 00:00:00,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,2020-02-01 00:00:00,2020-02-02 00:00:00,2020-02-03 00:00:00,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-08 00:00:00,...,2020-07-23 00:00:00,2020-07-24 00:00:00,2020-07-25 00:00:00,2020-07-26 00:00:00,2020-07-27 00:00:00,2020-07-28 00:00:00,2020-07-29 00:00:00,2020-07-30 00:00:00,2020-07-31 00:00:00,2020-08-01 00:00:00,2020-08-02 00:00:00,2020-08-03 00:00:00,2020-08-04 00:00:00,2020-08-05 00:00:00,2020-08-06 00:00:00,2020-08-07 00:00:00,2020-08-08 00:00:00,2020-08-09 00:00:00,2020-08-10 00:00:00,2020-08-11 00:00:00,2020-08-12 00:00:00,2020-08-13 00:00:00,2020-08-14 00:00:00,2020-08-15 00:00:00,2020-08-16 00:00:00,2020-08-17 00:00:00,2020-08-18 00:00:00,2020-08-19 00:00:00,2020-08-20 00:00:00,2020-08-21 00:00:00,2020-08-22 00:00:00,2020-08-23 00:00:00,2020-08-24 00:00:00,2020-08-25 00:00:00,2020-08-26 00:00:00,2020-08-27 00:00:00,2020-08-28 00:00:00,2020-08-29 00:00:00,2020-08-30 00:00:00,2020-08-31 00:00:00
new_deaths_Pakistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,32,54,24,35,20,23,27,32,27,0,0,33,15,15,21,17,16,14,15,15,17,10,14,9,6,7,15,0,19,10,12,4,0,20,12,7,9,1,4,6


In [ ]:
df.columns = ['date','new_cases_Pakistan']
df.dropna(inplace=True)
case_sequence = df['new_cases_Pakistan'].tolist()
len(case_sequence)

240

In [ ]:
df_deaths.columns = ['date','new_deaths_Pakistan']
df_deaths.dropna(inplace=True)
death_sequence=df_deaths['new_deaths_Pakistan'].tolist()
len(death_sequence)

240

In [ ]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
#raw_seq = [1748, 1356, 1446, 2076, 2801, 2429, 3039, 2964, 3938]
case_array=array(case_sequence)
death_array=array(death_sequence)
# choose a number of time steps
n_steps = 4
case_array = case_array.reshape((len(case_array), 1))
death_array = death_array.reshape((len(death_array), 1))
dataset = hstack((case_array, death_array))
# convert into input/output
X, y = split_sequences(dataset, n_steps)
n_features = X.shape[2]
print(X.shape, y.shape)
# summarize the data
# split into samples
#X, y = split_sequence(sequence, n_steps)
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
#n_features = 1
#n_seq = 2
#n_steps = 2
#X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

(236, 4, 2) (236, 2)


In [ ]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(3, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(20, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (None, 3, 4, 64)          192       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 3, 2, 64)          0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 3, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 50)             35800     
_________________________________________________________________
lstm_3 (LSTM)                (None, 20)                5680      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 42        
Total params: 41,714
Trainable params: 41,714
Non-trainable params: 0
__________________________________________________

In [ ]:
# fit model
model.fit(X, y, epochs=400, verbose=0)

ValueError: ignored

In [ ]:
# demonstrate prediction
x_input = array([1176, 936,	1063, 1114])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[1025.8849]]
